This notebook is written by the DAWS2 UQ team based in Univ. of Liverpool for the workshop of UQ & M of the DAWS2 project.

Uncertainty Quantification & Management
--
*placeholder texts*

{

Personal website: https://yuchenakaleslie.github.io/<br>
ResearchGate: https://www.researchgate.net/profile/Yu-Chen-402<br>
Contact: yu.chen2@liverpool.ac.uk

}

DAWS2 UQ team: {Scott Ferson, Ioanna,  [(Leslie) Yu Chen](https://yuchenakaleslie.github.io/)}

All rights reserved.

===================

## Table of Contents
#### [1. Setup](#Setup)
#### [2. Problem statement](#problem_statement)
#### [3. Create propagating function](#create_propagating_function)
#### [4. Epistemic uncertainty](#propagate_epistemic_uncertainty)
#### <small>[4.1. Uncertainty Characterisation](#4.1-uncertainty-characterisation)</small>
#### <small>[4.2. Uncertainty propagation methods](#4.2-uncertainty-propatation-methods)</small>
##### <small>[4.2.1. Endpoints propagation](#4.2.1-Endpoints-propatation)</small>
##### <small>[4.2.2. Subinterval reconstitution propagation](#4.2.2.-Subinterval-reconstitution-propagation)</small>
##### <small>[4.2.3. Sampling propagation](#4.2.3.-Sampling-propagation)</small>
##### <small>[4.2.4. Optimisation methods](#4.2.3.-Optimisation-methods)</small>
#### [5. Aleatory uncertainty](#aleatory_uncertainty_propagation)
#### [5. Mixed types of uncertainty](#mixed_uncertainty_propagation)

***
<a id="setup"></a>
# 1. Setup

Import the libraries

In [2]:
import numpy as np
from PyUncertainNumber.propagation.uncertaintyPropagation import Propagation, epistemic_propagation, aleatory_propagation, mixed_propagation
from PyUncertainNumber import UncertainNumber as UN
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pyuncertainnumber'

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext rich

In [19]:
# import user-defined function

USER_dir = 'C:\\Users\\Ioanna\\Documents\\GitHub\\PyUncertainNumber\\cantilever_beam'
import sys
sys.path.append(USER_dir)

***
<a id="problem_statement"></a>
# 2. Problem statement

The numerical example models a simple cantilever beam with length, $L$, distance to the neutral axis $y$, Young’s modulus, $E$, second moment of inertia, $I$, and external load, $F$. We will compute the bending stress $σ$, and the deflection $d$, assuming the above input parameters are **uncertain numbers**.

We will use this example throughout the document to illustrate the differences among various uncertainty quantification approaches. A complete description of the following methods can be found in the [DAWS1 report](https://sites.google.com/view/dawsreports/up).

![alt text](../assets/cantilever.png)

**Fig.1** Cantilever beam with input parameters.  

***
<a id="create_propagating_function"></a>
# 3. Create the propagating function


A propagating function is created which calculates the deflection, $d$, as a function of $L$, $E$, $I$, and $F$.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this.


Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.

<a id="3.1.-create-function"></a>

## 3.1. Create propagating function

In [3]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return np.array([deflection])

In [ ]:
# know the function with default Python help mechanism
cantilever_beam_deflection?

In [ ]:
import numpy as np 
x = np.array([1.00500000e+01 , 3.86159100e-04 ,-8.24387293e+00 , 2.12872340e+02])
y = cantilever_beam_deflection(x)
print(y)

<a id="3.2.-Upload-function"></a>

## 3.2. Upload propagating function

In [36]:
# Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.
from PyUncertainNumber.propagation.performance_func import cb_deflection, cb_stress

<a id="3.3.-verify-function"></a>

## 3.3. Verify the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [ ]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

***
<a id="propagate_epistemic_uncertainty"></a>
# 4. Epistemic uncertainty

<a id="4.1.-uncertainty-characterisation"></a>

## 4.1. Uncertainty Characterisation

Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below

- $y = [0.145, 0.155] \ m$

- $L = [9.95, 10.05] \ m$

- $I = [0.0003861591, 0.0005213425] \ m^{4}$

- $F = [11, 37] \ kN$

- $E = [200, 220] \ GPa$
 

In [4]:
#y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

In [ ]:

L.display()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_interval_box(lower_bound, upper_bound):
    """Plots an interval as a box on the y-axis at y=1 with no x-axis numbers 
       and includes vertical lines from y=0 to y=1 at the bounds.
    """

    plt.figure(figsize=(3, 3))  # Adjust figure size as needed

    # Create the box
    
    plt.hlines(y=1, xmin=lower_bound, xmax=upper_bound, linewidth=2, color='blue')

    # Add vertical lines
    plt.vlines(x=lower_bound, ymin=0, ymax=1, linewidth=1, color='blue')
    plt.vlines(x=upper_bound, ymin=0, ymax=1, linewidth=1, color='blue')

    #plt.yticks([0])  # Show only the y-axis tick at 0 
    plt.xticks([])  # Remove x-axis ticks and numbers

    plt.xlabel("V")
    plt.ylabel("CDF")

    plt.show()


# Example usage:
plot_interval_box(2, 5)

<a id="4.2-uncertainty-propatation-methods"></a>

## 4.2 Ucertainty propagation methods 

Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

<a id="4.2.1.-Endpoints-propatation"></a>

### 4.2.1. Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

For the working example, there are d = 5 intervals which results in $n = 2^{5} = 32$ input combinations.

#### Assumptions




- [x] Ioanna's original top-level UP function with  the 'endspoints' method

- [x] Leslie's standalone endspoints implementation

In [ ]:
METHOD = "endpoints"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )                 

In [ ]:
a.print() #summary of results 

In [ ]:
a.raw_data['x'] # the input combinations.
a.raw_data['f'] # the function evaluations for input combinations.
a.raw_data['bounds'] # the bounds for each output
a.raw_data['min'] # summary of min f and corresponding x
a.raw_data['max'] # summary of max f and corresponding x
a.un # the uncertain number


In [ ]:
METHOD = "endpoints"

b = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          n = None, 
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

In [ ]:
# returned a dict that contains the UN object and some other information
b.keys()

In [ ]:
b.raw_data['x'] # the input combinations.
b.raw_data['f'] # the function evaluations for input combinations.
b.raw_data['bounds'] # the bounds for each output
b.raw_data['min'] # summary of min f and corresponding x
b.raw_data['max'] # summary of max f and corresponding x
b.un # the uncertain number

In [ ]:
b.print()

<a id="4.2.2.-Subinterval-reconstitution-propagation"></a>

### 4.2.2. Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [ ]:
METHOD = "subinterval"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

In [ ]:
a.print() #summary of results 

In [ ]:
METHOD = "subinterval"

b = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

In [ ]:
b.print() #summary of results 

In [ ]:
b.raw_data['x'] # the input combinations.
b.raw_data['f'] # the function evaluations for input combinations.
b.raw_data['bounds'] # the bounds for each output
b.raw_data['min'] # summary of min f and corresponding x
b.raw_data['max'] # summary of max f and corresponding x
b.un # the uncertain number

<a id="4.2.3.-Sampling-propagation"></a>

### 4.2.3. Sampling propagation

- Brute Monte Carlo
- Latin Hypercube
- Brute Monte Carlo + endpoints
- Latin Hypercuve + endpoints

In [ ]:
METHOD = "monte_carlo"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

In [ ]:
a.pritn()

In [ ]:
METHOD = "monte_carlo"

b = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

In [ ]:
b.print()
b.un

In [ ]:
METHOD = "monte_carlo_endpoints"

c = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

In [ ]:
c.print()
c.un

In [ ]:
METHOD = "monte_carlo_endpoints"

d = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

In [ ]:
d.un

### 4.2.4. Sampling propagation

- Latin Hypercube
- Latin Hypercuve + endpoints

In [ ]:
METHOD = "latin_hypercube"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

In [ ]:
a.un

In [ ]:
METHOD = "latin_hypercube"

b = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

In [ ]:
b.un

In [ ]:
METHOD = "latin_hypercube_endpoints"

c = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )       

In [ ]:
c.un

In [ ]:
METHOD = "latin_hypercube_endpoints"

d = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )      

In [ ]:
d.print()

<a id="4.2.5.-Optimisation-methors"></a>

### 4.2.5. Optimisation methods

- Local optimisation
- Genetic algorithm


In [ ]:
METHOD = "local_optimisation"

a = Propagation(vars=[L, I, F, E], 
       fun = cantilever_beam_deflection, 
       x0 = None, 
       method = METHOD, 
       method_loc = 'Nelder-Mead')

In [ ]:
a.un
a.print()

In [10]:
METHOD = "local_optimisation"

b = epistemic_propagation(vars=[L, I, F, E], 
       fun = cantilever_beam_deflection, 
       #x0 = None, 
       method = METHOD, 
       method_loc = 'Nelder-Mead')

In [ ]:
b.print()

In [ ]:
b.un

In [ ]:
METHOD = "genetic_optimisation"

a = Propagation(vars=[L, I, F, E], 
       fun = cantilever_beam_deflection, 
       method = METHOD)

In [ ]:
a.un

In [ ]:
len(a.raw_data['bounds'].shape)

In [ ]:
a.raw_data['bounds']

In [ ]:
a.print()

In [ ]:
METHOD = "genetic_optimisation"

b = epistemic_propagation(vars[L, I, F, E], 
       fun = cantilever_beam_deflection, 
       method = METHOD)

In [ ]:
b.print()

### Naive interval arithmetics

In [ ]:
defl = cantilever_beam_deflection(L, I, F, E)

***
<a id="aleatory_uncertainty_propagation"></a>
# 5. Aleatory uncertainty

when inputs have various types of uncertainty.

In [17]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])

In [ ]:
METHOD = "monte_carlo"

a = Propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          n_sam = 100, 
          method = METHOD, 
          save_raw_data = "no"
         )


In [ ]:
a.print()

In [ ]:

METHOD = "monte_carlo"

b = aleatory_propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          n_sam = 100, 
          method = METHOD, 
          save_raw_data = "no"
         )


In [ ]:
b.print()

***
<a id="mixed_uncertainty_propagation"></a>
# 6. Mixed types of uncertainty

when inputs have various types of uncertainty.

In [ ]:
#TODO could Scott elaborate? cartesian matrix distribution and interval or pbox - interval horizontal slices through the pbox define an interval, horizontal slicing as an approximation or rigolously each of the ci functions
#TODO ferson, hajagos rigorous
#slicing approimately, slicling rigorously
#a list of itnervals by horizontal slices and with these lists the cartesian product , interval from each input.
#stack the intervals and reconstruct this is done. 

In [13]:
# y = UN(name='distance to neutral axis', symbol='y', units='m', essence='distribution', distribution_parameters=["gaussian", [0.15, 0.00333]])
# L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
    # E = N(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])
    
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) 
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
# I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
# F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])


In [ ]:
METHOD = "focused_discretisation_endpoints"

a = Propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          # n_disc = 10,
          
          # tOp = 0.999,
          # bOt= 0.001,
          # condensation = None, 
          method = METHOD, 
          save_raw_data = "no"
         )

In [ ]:
METHOD = "focused_discretisation_endpoints"

a = mixed_propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          # n_disc = 10,
          
          # tOp = 0.999,
          # bOt= 0.001,
          # condensation = None, 
          method = METHOD, 
          save_raw_data = "no"
         )

In [ ]:
METHOD = "focused_discretisation_extremepoints"

a = Propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          n_disc = 10,
         
          # tOp = 0.999,
          # bOt= 0.001,
          # condensation = None, 
          method = METHOD, 
          save_raw_data = "no"
         )


In [ ]:
a.print()


In [ ]:
a.raw_data['bounds']

In [ ]:
METHOD = "focused_discretisation_extremepoints"

a = mixed_propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          # n_slices = 10,
          # condensation = None, 
          # tOp = 0.999,
          # bOt= 0.001,
          method = METHOD, 
          save_raw_data = "no"
         )


In [ ]:
a.print()
a.bounds

#Approximation needs only distributions or pboxes. For the time being lets not use any intervals. 

In [24]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])

In [ ]:
METHOD = "varied_discretisation_extremepoints"

a = Propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          # n_slices = 10,
          # condensation = None, 
          # tOp = 0.999,
          # bOt= 0.001,
          method = METHOD, 
          save_raw_data = "no"
         )

In [ ]:
a.print()
a.un

In [ ]:
a.raw_data['x'] # the input combinations.
a.raw_data['f'] # the function evaluations for input combinations.
a.raw_data['bounds'] # the bounds for each output
a.raw_data['min'] # summary of min f and corresponding x
a.raw_data['max'] # summary of max f and corresponding x
a.un # the uncertain number

In [ ]:
METHOD = "varied_discretisation_extremepoints"

b = mixed_propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          # n_slices = 10,
          # condensation = None, 
          # tOp = 0.999,
          # bOt= 0.001,
          method = METHOD, 
          save_raw_data = "no"
         )

In [ ]:
b.print()

In [ ]:
b.raw_data['x'] # the input combinations.
b.raw_data['f'] # the function evaluations for input combinations.
b.raw_data['bounds'] # the bounds for each output
b.raw_data['min'] # summary of min f and corresponding x
b.raw_data['max'] # summary of max f and corresponding x
b.un # the uncertain number